# **COVID-19 - EXPLORATORY DATA ANALYSIS AND VISUALIZATION**

This kernel is created together with [another kernel](https://www.kaggle.com/nhntran/covid-19-vietnam-data-eda-and-visualization?scriptVersionId=32963257) which is for specific country data in Vietnam.

The writen up report for this analysis could be found at my blog (coming soon).

*Data used in this kernel: Updated data from the Johns Hopkins University (the CSSEGI)*


**REFERENCE SOURCES**

1. Kaggle:

https://www.kaggle.com/abhinand05/covid-19-digging-a-bit-deeper

https://www.kaggle.com/corochann/covid-19-eda-with-recent-update-on-april

https://www.kaggle.com/vikassingh1996/coronavirus-an-exploratory-study-w-detail-report/notebook

2. TowardsDataScience:

https://towardsdatascience.com/visualizing-the-coronavirus-pandemic-with-choropleth-maps-7f30fccaecf5

3. Other:
The data from https://github.com/CSSEGISandData/COVID-19

**A. ENVIRONMENT SETUP**

In [ ]:
# Install wget to download the data
!pip install wget

In [ ]:
# Import neccessary package
import os
import pandas as pd
import wget
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")

%config InlineBackend.figure_format = 'retina' #high resolution for rendered images on notebook

# Map visualization
import folium
import altair as alt

# Plotly
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True) # Required to use plotly offline in jupyter notebook
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_white"

'''Display markdown formatted output like bold, italic bold etc.'''
from IPython.display import Markdown
def bold(string):
    display(Markdown(string))

**B. DOWNLOADING THE DATA**

The updated data from the Johns Hopkins University (the CSSEGI)

In [ ]:
# Remove on the downloaded csv files (new data everyday from the CSSEGI)
! rm *.csv

# The updated data from the Johns Hopkins University:
# global confirm cases and death cases:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv']

for url in urls:
    filename = wget.download(url)
# data will be in /kaggle/working folder

In [ ]:
# Read the dataset
cases_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('time_series_covid19_deaths_global.csv')
cases_df.head()

In [ ]:
death_df.head()

**C. DATA WRANGLING - DATA EXPLORATORY ANALYSIS**

**1. General information about the data**

In [ ]:
## General information about the data

# print(cases_df.shape)
# print("All columns:", cases_df.columns)
# print("Types:", cases_df.dtypes)

# cases_df.describe(include='all')

# ## Missing values ?
# cases_df.info()
# cases_df.isnull().sum().sort_values(ascending=False)

# => Only missing value in Province/State

**2. Merging and cleaning up the data**

- Merge confirmed cases and death cases.
- Correct the values in 'Province/State' 
- Changing the country name for using the package 'pycoutry_convert'
- Fixing wrong data in a certain date in 'Hubei', China


In [ ]:
## Combine Confirmed and Death Cases:
dates = cases_df.columns[4:]
cases_df_cleanup = cases_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                                value_vars = dates, var_name = 'Date', value_name = 'Confirmed Cases')
death_df_cleanup = death_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                                value_vars = dates, var_name = 'Date', value_name = 'Deaths')
data = pd.concat([cases_df_cleanup, death_df_cleanup['Deaths']], axis = 1, sort = False)
data.head()

In [ ]:
data.columns

In [ ]:
len(data['Province/State'].unique())

In [ ]:
## Correct the values in 'Province/State' 
##(remove the 'Recovered' values and the "," in 'Bonaire, Sint Eustatius and Saba' to avoid redundancy)

data = data[data['Province/State'].str.contains('Recovered') != True]
data = data[data['Province/State'].str.contains(',') != True]

In [ ]:
len(data['Province/State'].unique())

In [ ]:
len(data['Country/Region'].unique())

In [ ]:
## Checking some country names
data[data['Country/Region'] == 'Taiwan*']
#data[data['Country/Region'] == "Cote d'Ivoire"]

In [ ]:
## Changing the country name for using the package 'pycoutry_convert'
# Dict to change:
country = {'US':'USA', 
           'Korea, South':'South Korea',
           'Taiwan*': 'Taiwan',
           'Congo (Kinshasa)': 'Democratic Republic of the Congo',
           "Cote d'Ivoire": "Côte d'Ivoire",
           'Reunion': 'Réunion',
           'Congo (Brazzaville)': 'Republic of the Congo',
           'Bahamas, The': 'Bahamas',
           'Gambia, The': 'Gambia'
          }
for old, new in country.items():
    data['Country/Region'] = data['Country/Region'].replace(old, new)

In [ ]:
## Checking the result of changing some country names
#data[data['Country/Region'] == 'Taiwan']
data[data['Country/Region'] == "Bahamas"]

In [ ]:
len(data['Country/Region'].unique())

In [ ]:
## Show all of the data from China
#data.loc[data['Country/Region'] == 'China']

## Show the specific data that is wrong (according to literature)
data[(data['Province/State'] == 'Hubei') & (data['Date'] == '2/12/20')] 

In [ ]:
## Fixing wrong data in a certain date in 'Hubei', China

## fixing function
def fixing_value(date, region, value_name, new_value):
    for key, val in new_value.items():
        data.loc[(data['Date'] == date) & (data[region] == key), value_name] = val
# fix data       
hubei_feb12 = {'Hubei':34874}
fixing_value('2/12/20', 'Province/State', 'Confirmed Cases', hubei_feb12)

#checking the fixing effect
data[(data['Province/State'] == 'Hubei') & (data['Date'] == '2/12/20')] 

In [ ]:
### Convert date from string to datetime type

data['Date'] = pd.to_datetime(data['Date'])
# need to convert 'Date' data from string to datetime,
# sort by date again. Otherwise 2/1/20 then 2/10/20 => wrong sorting
#data['Date'] = data['Date'].dt.strftime('%m/%d/%Y')
data['Date'] = data['Date']
data.head()

**3. Saving the data for further analysis**

In [ ]:
data.to_csv('global_covid19cases.csv', index = False)

**4. Extracting specific data for each country**

In [ ]:
## Show all of the data from US
#data.loc[data['Country/Region'] == 'US']

## Extract the specific country data
us_data = data[data['Country/Region'] == 'US']
us_data

**D. WORLD WIDE DETAIL REPORT ON THE MOST UPDATED DAY (SINGLE DAY)**

**1. The globally confirmed cases and deaths**

In [ ]:
## Extract the final date in the dataframe
date = data['Date'].iloc[-1]

most_recent_data = data[data['Date'] == date]
print('Globally COVID-19 data on date {}:\n'.format(date))
print('Confirmed Cases:   {:,}'.format(most_recent_data['Confirmed Cases'].sum()))
print('Deaths Cases:      {:,}'.format(most_recent_data['Deaths'].sum()))

**2. Distribution of cases by country**

In [ ]:
color_case = 'YlOrRd'
color_death = 'YlOrRd'
# color reference
#cmaps['Sequential'] = [
#            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
#            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
#            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

country_cases = most_recent_data.groupby('Country/Region')['Confirmed Cases', 'Deaths'].sum().reset_index()
country_cases.sort_values('Confirmed Cases', ascending = False)\
            .style.background_gradient(cmap = color_case, subset = ['Confirmed Cases'])\
            .background_gradient(cmap = color_death, subset = ['Deaths'])

**3. Graph presentation top 10 countries**

In [ ]:
## SINGLE VALUE GRAPH (** Using altair (alt) for this graph)
## list of parameters
bar_color = '#da635eff'
sort_value = 'Confirmed Cases' # 'Deaths'/'Recovered'
y_axis = 'Country/Region'# 'Province/State'
top_num = 10 # number of country
threshold = 1000

## preparing data for graph (extracting top countries)
top10_country = country_cases.sort_values(sort_value, ascending = False).head(top_num)
# Can also choose not top_num but a cutoff threshold of sort_value
#top10_country = country_cases[country_cases[sort_value] > threshold].sort_values(sort_value, ascending = False)

## drawing graph
def drawing_single_value_bar_graph(data, bar_color, sort_value, y_axis, top_num):
    
    bars = alt.Chart(data)\
        .mark_bar(color = bar_color,cornerRadiusTopLeft = 3, cornerRadiusTopRight=3, size = 20, opacity = 0.7)\
        .encode(
                    x = '{}:Q'.format(sort_value),
                    y = alt.Y('{}:O'.format(y_axis), sort = '-x'))\
        .properties( 
            title = {
            "text":['Top {}: {}'.format(y_axis, sort_value)],
            "subtitle":['*Updated on {}'.format(date)],
            "fontSize":15,
            "fontWeight": 'bold',
            "font":'Courier New',
            }
        )
    # dx = 3 Nudges text to right so it doesn't appear on top of the bar
    text = bars.mark_text(align = 'left', baseline = 'middle',dx = 3).encode(text = '{}:Q'.format(sort_value))

    return (bars + text).properties( height = 400, width = 800)
    

fig = drawing_single_value_bar_graph(top10_country, bar_color, sort_value, y_axis, top_num)
fig

**4. Distribution of cases on map**

In [ ]:
## Using most_recent_data data
most_recent_data.head()

In [ ]:
##### *********  MAP - STYLE 1  *********
# Using folium
# Data: Using most_recent_data dataframe

## parameter for map
mapstyle = 'CartoDB positron'
line_color = '#da635eff'
fill_color = '#da635eff'
fill_opacity = 0.6
# other styles: 'OpenStreetMap', "Stamen Terrain”, “Stamen Toner”, “Stamen Watercolor”

## create map
world_map = folium.Map(location = [10,0], zoom_start = 2, max_zoom = 8, min_zoom = 2, tiles = mapstyle)
## define detail of the map
for lat, long, case, name in zip(most_recent_data['Lat'], most_recent_data['Long'], most_recent_data['Confirmed Cases'],\
                                most_recent_data['Country/Region']):
    folium.CircleMarker([lat, long], radius = (int((np.log(case+1.00001)))+0.2),
                       popup = ("<h5 style='text-align:center;font-weight: bold'>" + str(name).capitalize()+ "</h5>" + '<br>'
                                '<strong>Confirmed Cases</strong>: ' + str(case) + '<br>'),\
                       color = line_color, weight= 1.5, \
                        fill_color = fill_color, fill_opacity = fill_opacity).add_to(world_map)
# opacity = fill_opacity #opacity of the line 
 
## Save map
world_map.save("./world_map.html")
world_map


In [ ]:
##### *********  MAP - STYLE 2  *********
# Using choropleth
# Data: Using country_cases dataframe (log10 scale)

## list of parameters
color = '#da635eff' #'Reds'
map_value = 'Confirmed Cases' # 'Deaths'/'Recovered'

## function to drawing graph
def drawing_global_heatmap(country_cases, map_value, color):
    temp_df = country_cases[['Country/Region',map_value]]

    fig = px.choropleth(temp_df, locations="Country/Region",
                        color = np.log10(temp_df[map_value] + 1), # + 1 to avoid divided by 0, log0
                        hover_name = "Country/Region", # column to add to hover information
                        hover_data = [map_value],
                        color_continuous_scale = px.colors.sequential.Plasma,locationmode = "country names")
    fig.update_geos(fitbounds = "locations", visible = False)
    fig.update_layout(title_text = "{} Heat Map (Log Scale)".format(map_value), title_x = 0.5)
    fig.update_coloraxes(colorbar_title = "{}(Log Scale)".format(map_value),colorscale="Reds")
    return fig
## drawing graph
fig = drawing_global_heatmap(country_cases, map_value, color)
# fig.to_image("Global Heat Map {map_value}.png")
fig.show()

In [ ]:
## drawing graph
fig = drawing_global_heatmap(country_cases,'Deaths', 'Reds')
# fig.to_image("Global Heat Map {map_value}.png")
fig.show()

**E. WORLD WIDE DETAIL REPORT SINCE THE EARLY STAGE OF THE PANDEMIC - UP TO NOW**

In [ ]:
## Using datafram 'data'
data.head()
# Note: this dataframe 'data' has detail data of some province/state in the same country.
# If analyzing country data: Must use groupby('Country/Region') 
# to sum the cases of all the Province/State of the same country, 
# for example, Canada:
data[data['Country/Region'] == "Canada"]

**1. Static graph**

In [ ]:
## Sum up all the cases in the world by date
world_cases_all_time = data.groupby('Date')['Confirmed Cases', 'Deaths'].sum().sort_values('Date').reset_index()

# Adding 'New Confirmed Cases' column
world_cases_all_time['New Confirmed Cases'] = world_cases_all_time['Confirmed Cases'] - world_cases_all_time['Confirmed Cases'].shift(1)
# # Adding 'Mortality' column
# world_cases_all_time['Mortality'] = world_cases_all_time['Deaths']/world_cases_all_time['Confirmed Cases']

# Checking the adding
world_cases_all_time.head()
#world_cases_all_time.tail()


In [ ]:
## For good x-axis display (only show day and year)
# world_cases_all_time['Date'] = pd.to_datetime(world_cases_all_time['Date'])
# world_cases_all_time['Date'] = world_cases_all_time['Date'].dt.strftime('%m/%d')
##### *********  GRAPH - STYLE 1 - LINE  *********

# Combine all the data together for drawing graph
world_cases_all_time_melt = world_cases_all_time.melt(id_vars = ['Date'], 
                                    value_vars = ['Confirmed Cases', 'Deaths', 'New Confirmed Cases'])
#world_cases_all_time_melt.head()

fig = px.line(world_cases_all_time_melt, x="Date", y="value", color='variable')
fig.update_layout(title = {'text': 'Worldwide Confirmed/Death Cases Over Time', 'x': 0.5},
                   xaxis_title = 'Date (2020)',
                   yaxis_title = 'Cases',
                 legend = {'title': None})
#fig.update_xaxes(dtick = 10) # changing the distance between ticks
fig.show()

In [ ]:
##### *********  GRAPH - STYLE 2 - BAR  *********
# Using the dataframe world_cases_all_time
# 'Date', Confirmed Cases', 'Deaths', 'New Confirmed Cases'

fig = go.Figure(data=[
    go.Bar(name = 'Confirmed Cases', x = world_cases_all_time['Date'], y = world_cases_all_time['Confirmed Cases']),
    go.Bar(name = 'Deaths', x = world_cases_all_time['Date'], y = world_cases_all_time['Deaths'])
])

fig.update_layout(title = {'text': 'Worldwide Confirmed/Death Cases Over Time', 'x': 0.5},
                   xaxis_title = 'Date (2020)',
                   yaxis_title = 'Cases',
                 legend = {'title': None})
fig.show()

**2. Animated graph**

In [ ]:
## Sum up all the cases for each country by date
countries_cases_all_time = data.groupby(['Date', 'Country/Region'])['Confirmed Cases', 'Deaths'].sum().sort_values('Date').reset_index()

# checking the grouping
countries_cases_all_time.tail()

In [ ]:
world_cases_all_time

In [ ]:

##### *********  MAP - STYLE 2 WITH ANIMATION  *********
# Using choropleth
# Data: Using country_cases dataframe (log10 scale)

## list of parameters
color = '#da635eff' #'Reds'
map_value = 'Confirmed Cases' # 'Deaths'/'Recovered'

## function to drawing graph
temp_df = countries_cases_all_time[['Date', 'Country/Region','Confirmed Cases']]
temp_df['Date'] = temp_df['Date'].dt.strftime('%m/%d/%Y')
fig = px.choropleth(temp_df, locations = "Country/Region", locationmode = 'country names', 
                     color = np.log10(temp_df[map_value] + 1), 
                     hover_name = "Country/Region", projection="mercator",
                     animation_frame = "Date", width = 1000, height = 800,
                     color_continuous_scale = px.colors.sequential.Viridis,
                     title = 'The Spread of COVID-19 Cases Across The World')

#Showing the figure
#fig.update_geos(fitbounds = "locations", visible = False)
fig.update(layout_coloraxis_showscale=True)
fig.update_coloraxes(colorbar_title = "{} (Log Scale)".format(map_value), colorscale = "Reds")
py.offline.iplot(fig)
# fig.to_image("Global Heat Map {map_value}.png")

**F. COUNTRY COMPARISON**

In [ ]:
## Sum up all the cases for each country by date
countries_cases_all_time = data.groupby(['Date', 'Country/Region'])['Confirmed Cases', 'Deaths'].sum().sort_values('Date').reset_index()

# checking the grouping
countries_cases_all_time.tail()

In [ ]:
top10_country = country_cases.sort_values(sort_value, ascending = False).head(10)['Country/Region'].unique()
top10_country_cases = countries_cases_all_time[countries_cases_all_time['Country/Region'].isin(top10_country)]
top10_country_cases

In [ ]:
## Graph top 10 country, y = confirmed case
fig = px.line(top10_country_cases,
              x='Date', y='Confirmed Cases', color='Country/Region',
              title=f'Confirmed Cases for top 10 country')
fig.update_layout(legend = {'title': None})
#fig.update_xaxes(dtick = 10)
fig.show()

In [ ]:
## Graph top 10 country, y = log(confirmed case)
fig = px.line(top10_country_cases,
              x='Date', y='Confirmed Cases', color='Country/Region',
              title=f'Confirmed Cases for Top 10 Country')
fig.update_layout(legend = {'title': None},
                 xaxis_title = 'Date', yaxis_title = 'Log(Cases)')
#fig.update_xaxes(dtick = 10)
fig.update_layout(yaxis_type="log")
fig.show()

In [ ]:
target_countries = np.array(['USA', 'United Kingdom', 'Spain', 'China', 'Italy', 'Germany',
                             'Singapore', 'Japan','South Korea', 'Vietnam'])
target_country_cases = countries_cases_all_time[countries_cases_all_time['Country/Region'].isin(target_countries)]
fig = px.line(target_country_cases,
              x = 'Date', y = 'Confirmed Cases', color = 'Country/Region',
              title = f'Confirmed Cases for target countries')
fig.update_layout(legend = {'title': None})
#fig.update_xaxes(dtick = 10)
fig.show()

**1. COMPARING DIFFERENT COUNTRIES - DAYS SINCE 50TH CASES**

Using the list of target_countries

In [ ]:
target_countries = np.array(['USA', 'United Kingdom', 'Spain', 'China', 'Italy', 'Germany',
                             'Singapore', 'Japan','South Korea', 'Vietnam'])
target_country_cases = countries_cases_all_time[countries_cases_all_time['Country/Region'].isin(target_countries)]

In [ ]:
## Comparison since 50 cases in each country

threshold = 50 # 50 cases
countries_since_50cases = pd.DataFrame(columns = ['Date', 'Country/Region',
                                                 'Confirmed Cases', 'Deaths', 'Date Since 50 Cases', 'Confirmed Cases (log10)'])
for country_name in target_countries:
    # Extract country data:
    country = target_country_cases[target_country_cases['Country/Region'] == country_name]
    country = country[country['Confirmed Cases'] >= threshold]
    start_date = country['Date'].min()
    country['Date Since 50 Cases'] = (country['Date'] - start_date)/pd.Timedelta('1 days')
    country['Confirmed Cases (log10)'] = np.log10(country['Confirmed Cases'])
    countries_since_50cases = countries_since_50cases.append(country)
countries_since_50cases

In [ ]:
### HOW NUMBER OF COVID-19 CASES INCREASES SINCE 50 CASES IN EACH COUNTRY

## Draw graph for different target countries:
fig = px.line(countries_since_50cases,
              x = 'Date Since 50 Cases', y = 'Confirmed Cases (log10)', color = 'Country/Region',
             hover_name = 'Country/Region', hover_data = ['Date Since 50 Cases', 'Confirmed Cases'])
## Update layout
fig.update_layout(title = {'text': '<b>Number of Confirmed Cases by Country Since 50 Cases</b>',
                           'x': 0.5},
                   xaxis_title = '<b>Days Since 50 Cases</b>',
                   yaxis_title = '<b>Confirmed Cases</b>',
                     legend = {'title': None})

## Add reference lines
# Case doubles every day
x1 = np.arange(0, 15)
y1 = np.log10(2**(x1 + np.log2(50)))
fig.add_trace(go.Scatter(x = x1, y = y1, mode='lines',
                         name = 'Case doubles every day',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))
# Case doubles every 3 days
x3 = np.arange(0, 50)
y3 = np.log10(2**(x3/3 + np.log2(50)))
fig.add_trace(go.Scatter(x = x3, y = y3, mode='lines',
                         name = 'Case doubles every 3 days',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))

# Case doubles every 6 days
x6 = np.arange(0, 80)
y6 = np.log10(2**(x6/6 + np.log2(50)))
fig.add_trace(go.Scatter(x = x6, y = y6, mode='lines',
                         name = 'Case doubles every 6 days',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))

# Case doubles every 2 weeks
x14 = np.arange(0, 80)
y14 = np.log10(2**(x14/14 + np.log2(50)))
fig.add_trace(go.Scatter(x = x14, y = y14, mode='lines',
                         name = 'Case doubles every 2 weeks',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))

## Add annotation for reference lines 
fig.update_layout(
    annotations = [
        dict(
            x = x1[-1],
            y = y1[-1],
            text = "Case doubles every day"
        ),
        dict(
            x = x3[-1],
            y = y3[-1],
            text = "Case doubles every 3 days"
        ),
        dict(
            x = x6[-1],
            y = y6[-1],
            text = "Case doubles every 6 days"
        ),
        dict(
            x = x14[-1],
            y = y14[-1],
            text = "Case doubles every 2 weeks"
        )
    ]
)

## Set y-axis to real number of cases (instead of log(cases)) to be more clear.
fig.update_yaxes(
    ticktext=["100 ", "1K ", "10K ", "100K ", "1M "],
    tickvals=[2, 3, 4, 5, 6],
)

fig.show()

**2. COMPARING COUNTRY SINCE 100 CONFIRMED CASES**

In [ ]:
## Comparison since 100 cases in each country

threshold = 100 # 100 cases
countries_since_100cases = pd.DataFrame(columns = ['Date', 'Country/Region',
                                                 'Confirmed Cases', 'Deaths', 'Date Since 100 Cases', 'Confirmed Cases (log10)'])

# Record information for text label to the right end of every trace
x_pos = []
y_pos = []
label = []
for country_name in target_countries:
    # Extract country data:
    label.append(country_name)
    country = target_country_cases[target_country_cases['Country/Region'] == country_name]
    country = country[country['Confirmed Cases'] >= threshold]
    start_date = country['Date'].min()
    country['Date Since 100 Cases'] = (country['Date'] - start_date)/pd.Timedelta('1 days')
    country['Confirmed Cases (log10)'] = np.log10(country['Confirmed Cases'])
    x_pos.append(country['Date Since 100 Cases'].iloc[-1] + 2)
    y_pos.append(country['Confirmed Cases (log10)'].iloc[-1])
    countries_since_100cases = countries_since_100cases.append(country)
countries_since_100cases

In [ ]:
### HOW NUMBER OF COVID-19 CASES INCREASES SINCE 100 CASES IN EACH COUNTRY


## Draw graph for different target countries:
fig = px.line(countries_since_100cases,
              x = 'Date Since 100 Cases', y = 'Confirmed Cases (log10)', color = 'Country/Region',
              color_discrete_sequence = px.colors.qualitative.Pastel1,
             hover_name = 'Country/Region', hover_data = ['Date Since 100 Cases', 'Confirmed Cases'])
## Update layout
fig.update_layout(title = {'text': '<b>Number of Confirmed Cases by Country Since 100 Cases</b>',
                           'x': 0.5},
                   xaxis_title = '<b>Days Since 100 Cases</b>',
                   yaxis_title = '<b>Confirmed Cases</b>',
                     legend = {'title': None})

## Add reference lines
# Case doubles every day
x1 = np.arange(0, 15)
y1 = np.log10(2**(x1 + np.log2(100)))
fig.add_trace(go.Scatter(x = x1, y = y1, mode='lines',
                         name = 'Case doubles every day',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))
# Case doubles every 3 days
x3 = np.arange(0, 50)
y3 = np.log10(2**(x3/3 + np.log2(100)))
fig.add_trace(go.Scatter(x = x3, y = y3, mode='lines',
                         name = 'Case doubles every 3 days',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))

# Case doubles every 6 days
x6 = np.arange(0, 80)
y6 = np.log10(2**(x6/6 + np.log2(100)))
fig.add_trace(go.Scatter(x = x6, y = y6, mode='lines',
                         name = 'Case doubles every 6 days',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))

# Case doubles every 2 weeks
x14 = np.arange(0, 80)
y14 = np.log10(2**(x14/14 + np.log2(100)))
fig.add_trace(go.Scatter(x = x14, y = y14, mode='lines',
                         name = 'Case doubles every 2 weeks',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))

# Case doubles every 1 month
x30 = np.arange(0, 80)
y30 = np.log10(2**(x30/30 + np.log2(100)))
fig.add_trace(go.Scatter(x = x30, y = y30, mode='lines',
                         name = 'Case doubles every 1 month',
                         hoverinfo = "none", showlegend = False,
                         line = dict(dash ='dash', width = 2,
                                   color = ('rgb(200, 200, 200)'))))

## Add a text label to the right end of every trace. Most of the code below  
    # is adding specific offsets y position because some labels overlapped. 
# fig.add_trace(go.Scatter(
#     x = x_pos, y= y_pos, mode = 'text', text = label
# ))

## Add Vietnam trace
df_vietnam = countries_since_100cases[countries_since_100cases['Country/Region'] == 'Vietnam']

fig.add_trace(go.Scatter(x = df_vietnam['Date Since 100 Cases'], y = df_vietnam['Confirmed Cases (log10)'], mode='lines',
                         name = 'Vietnam',
                         hoverinfo = "none", showlegend = False,
                         line = dict(width = 4,
                                   color = ('red'))))
## Add annotation for reference lines 
fig.update_layout(
    annotations = [
        dict(
            x = x1[-1],
            y = y1[-1],
            text = "Case doubles every day"
        ),
        dict(
            x = x3[-1],
            y = y3[-1],
            text = "Case doubles every 3 days"
        ),
        dict(
            x = x6[-1],
            y = y6[-1],
            text = "Case doubles every 6 days"
        ),
        dict(
            x = x14[-1],
            y = y14[-1],
            text = "Case doubles every 2 weeks"
        ),
        dict(
            x = x30[-1],
            y = y30[-1],
            text = "Case doubles every 1 month"
        ),
        ## Reference line for some countries (manually adjust since overlapped)
        dict(
            x = df_vietnam['Date Since 100 Cases'].iloc[-1] + 8,
            y = df_vietnam['Confirmed Cases (log10)'].iloc[-1] + 0.02,
            xref="x",
            yref="y",
            text = "<b>Vietnam</b>",
            showarrow=False,align = 'right',
            font=dict(
            family="Courier New, monospace",
            size=16,
            color="red"
            ),
        )
    ]
)

## Set y-axis to real number of cases (instead of log(cases)) to be more clear.
fig.update_yaxes(
    ticktext=["100 ", "1K ", "10K ", "100K ", "1M "],
    tickvals=[2, 3, 4, 5, 6],
)

fig.show()

**MORE BEAUTIFUL GRAPH**

In [ ]:
end_ =  countries_since_100cases['Date Since 100 Cases'].max()
end_
new_df = pd.DataFrame(np.arange(0,end_ + 1))
new_df.columns = ['Date Since 100 Cases']
new_df 

In [ ]:
for country_name in target_countries:
    #Extract each country
    country = target_country_cases[target_country_cases['Country/Region'] == country_name]
    country = country[country['Confirmed Cases'] >= threshold]
    start_date = country['Date'].min()
    country['Date Since 50 Cases'] = (country['Date'] - start_date)/pd.Timedelta('1 days')
    country['Confirmed Cases (log10)'] = np.log10(country['Confirmed Cases'])
    # New column for each country
    new_df.join(country['Confirmed Cases (log10)'], lsuffix='_caller', rsuffix='_other')
    
new_df

**3. MORE INTERACTIVE GRAPH WITH HOVER FUNCTION - COMPARING COUNTRY SINCE 100 CONFIRMED CASES**

- Hightlight the country of interest on graph

In [ ]:
countries_since_100cases.head()

In [ ]:
### Need to separate each country as x and y
for country_name in target_countries:
    country = countries_since_100cases[countries_since_100cases['Country/Region'] == country_name]
    x = country['Date Since 100 Cases']
    y = country['Confirmed Cases (log10)']

In [ ]:
country_name = 'USA'
country = countries_since_100cases[countries_since_100cases['Country/Region'] == country_name]
x_USA = country['Date Since 100 Cases']
y_USA = country['Confirmed Cases (log10)']

In [ ]:
## NOT IN USE - JUST A REFERENCE
## function for a reference line
# def create_ref_line(threshold, num_days, x_limit, name):
#     x = np.arange(0, x_limit)
#     y = np.log10(2**(x/num_days + np.log2(threshold)))
#     ref_line = pd.DataFrame(columns = ['x', 'y','Name of Reference Line'])
#     ref_line['x'] = x
#     ref_line['y'] = y
#     ref_line['Name of Reference Line'] = name
#     return ref_line

# threshold = 100
# ## Add lines
# line1 = create_ref_line(threshold, 1, 15, 'Case doubles every day')
# line3 = create_ref_line(threshold, 3, 50, 'Case doubles every 3 days')
# line6 = create_ref_line(threshold, 6, 80, 'Case doubles every 6 days')
# line14 = create_ref_line(threshold, 14, 80, 'Case doubles every 2 weeks')
# line30 = create_ref_line(threshold, 30, 80, 'Case doubles every month')

In [ ]:
### HOW NUMBER OF COVID-19 CASES INCREASES SINCE 100 CASES IN EACH COUNTRY
## Using FigureWidget

threshold = 100 # 100 cases

# Reference line:
def create_line(threshold, num_days, x_limit):
    return [0, np.log10(2**(0 + np.log2(threshold))), x_limit, 
            np.log10(2**(x_limit/num_days + np.log2(threshold)))]
line1 = create_line(100, 1, 15) # 'Case doubles every day'
line3 = create_line(threshold, 3, 50) # 'Case doubles every 3 days'
line6 = create_line(threshold, 6, 80) # 'Case doubles every 6 days'
line14 = create_line(threshold, 14, 80) # 'Case doubles every 2 weeks'
line30 = create_line(threshold, 30, 80) # 'Case doubles every month'
    
fig = go.FigureWidget()
fig.layout.hovermode = 'closest'
fig.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2



## Update layout
fig.update_layout(title = {'text': '<b>Number of Confirmed Cases by Country Since 100 Cases</b>',
                           'x': 0.5},
                   xaxis_title = '<b>Days Since 100 Cases</b>',
                   yaxis_title = '<b>Confirmed Cases</b>',
                     legend = {'title': None})

## Add country data lines
annotation_list = [] #add annotation to be use, lists can contain different variable types
for country_name in target_countries:
    country = countries_since_100cases[countries_since_100cases['Country/Region'] == country_name]
    x = country['Date Since 100 Cases']
    y = country['Confirmed Cases (log10)']
    annotation_list.extend([x.iloc[-1], y.iloc[-1], country_name])
    fig.add_trace(go.Scatter(x = x, y = y, mode='lines',
                         name = country_name,
                        showlegend = False,
                         opacity = 0.8,
                         line = dict(dash = 'solid', width = 2,
                                   color = 'rgb(200, 200, 200)')))
fig.add_shape(
        # Case doubles every day
            type="line",
            x0 = line1[0],
            y0 = line1[1],
            x1 = line1[2],
            y1 = line1[3],
            line = dict(
                color = 'rgb(200, 200, 200)',
                width = 2,
                dash = "dash",
            )
)
fig.add_shape(
        # Case doubles every 3 days
            type="line",
            x0 = line3[0],
            y0 = line3[1],
            x1 = line3[2],
            y1 = line3[3],
            line = dict(
                color = 'rgb(200, 200, 200)',
                width = 2,
                dash = "dash",
            )
)
fig.add_shape(
        # Case doubles every 6 days
            type="line",
            x0 = line6[0],
            y0 = line6[1],
            x1 = line6[2],
            y1 = line6[3],
            line = dict(
                color = 'rgb(200, 200, 200)',
                width = 2,
                dash = "dash",
            )
)
fig.add_shape(
        # Case doubles every 14 days
            type="line",
            x0 = line14[0],
            y0 = line14[1],
            x1 = line14[2],
            y1 = line14[3],
            line = dict(
                color = 'rgb(200, 200, 200)',
                width = 2,
                dash = "dash",
            )
)
fig.add_shape(
        # Case doubles every 1 month
            type="line",
            x0 = line30[0],
            y0 = line30[1],
            x1 = line30[2],
            y1 = line30[3],
            line = dict(
                color = 'rgb(200, 200, 200)',
                width = 2,
                dash = "dash",
            )
)
## Add annotation for reference lines 
fig.update_layout(
    annotations = [
        dict(
            x = line1[2],
            y = line1[3],
            # xanchor="right",
            text = "Case doubles every day"
        ),
        dict(
            x = line3[2],
            y = line3[3],
            text = "Case doubles every 3 days"
        ),
        dict(
            x = line6[2],
            y = line6[3],
            text = "Case doubles every 6 days"
        ),
        dict(
            x = line14[2],
            y = line14[3],
            text = "Case doubles every 2 weeks"
        ),
        dict(
            x = line30[2],
            y = line30[3],
            text = "Case doubles every 1 month"
        )
    ]
)

## Set y-axis to real number of cases (instead of log(cases)) to be more clear.
# fig.update_yaxes(
#     ticktext=["100 ", "1K ", "10K ", "100K ", "1M "],
#     tickvals=[2, 3, 4, 5, 6],
# )

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one event they it be empty
    if len(points.point_inds) > 0:
        for i in range( len(fig.data) ):
            #fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)
            if i == points.trace_index:
                fig.data[i]['line']['color'] = 'red'
                fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta
            else:
                fig.data[i]['line']['color'] = 'rgb(200, 200, 200)'
                fig.data[i]['line']['width'] = default_linewidth
                
# we need to add the on_click event to each trace separately       
for i in range( len(fig.data) ):
    fig.data[i].on_click(update_trace)

# let's show the figure 
fig

In [ ]:
### NOT IN USE, JUST A REFERENCE - ADD COUNTRY NAME TO EACH LINE ON GRAPH
### HOW NUMBER OF COVID-19 CASES INCREASES SINCE 100 CASES IN EACH COUNTRY
## Using FigureWidget

# threshold = 100 # 100 cases

# # Reference line:
# def create_line(threshold, num_days, x_limit):
#     return [0, np.log10(2**(0 + np.log2(threshold))), x_limit, 
#             np.log10(2**(x_limit/num_days + np.log2(threshold)))]
# line1 = create_line(100, 1, 15) # 'Case doubles every day'
# line3 = create_line(threshold, 3, 50) # 'Case doubles every 3 days'
# line6 = create_line(threshold, 6, 80) # 'Case doubles every 6 days'
# line14 = create_line(threshold, 14, 80) # 'Case doubles every 2 weeks'
# line30 = create_line(threshold, 30, 80) # 'Case doubles every month'
    
# fig = go.FigureWidget()
# fig.layout.hovermode = 'closest'
# fig.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
# default_linewidth = 2
# highlighted_linewidth_delta = 2



# ## Update layout
# fig.update_layout(title = {'text': '<b>Number of Confirmed Cases by Country Since 100 Cases</b>',
#                            'x': 0.5},
#                    xaxis_title = '<b>Days Since 100 Cases</b>',
#                    yaxis_title = '<b>Confirmed Cases</b>',
#                      legend = {'title': None})

# ## Add country data lines
# annotation_list = [] #add annotation to be use, lists can contain different variable types
# x_list = []
# y_list = []
# for country_name in target_countries:
#     country = countries_since_100cases[countries_since_100cases['Country/Region'] == country_name]
#     x = country['Date Since 100 Cases']
#     y = country['Confirmed Cases (log10)']
#     annotation_list.append(country_name)
#     x_list.append(x.iloc[-1])
#     y_list.append(y.iloc[-1])
    
#     #draw line
#     fig.add_trace(go.Scatter(x = x, y = y, mode='lines',
#                          name = country_name,
#                         showlegend = False,
#                          opacity = 0.8,
#                          line = dict(dash = 'solid', width = 2,
#                                    color = 'rgb(200, 200, 200)')))

# fig.add_shape(
#         # Case doubles every day
#             type="line",
#             x0 = line1[0],
#             y0 = line1[1],
#             x1 = line1[2],
#             y1 = line1[3],
#             line = dict(
#                 color = 'rgb(200, 200, 200)',
#                 width = 2,
#                 dash = "dash",
#             )
# )
# fig.add_shape(
#         # Case doubles every 3 days
#             type="line",
#             x0 = line3[0],
#             y0 = line3[1],
#             x1 = line3[2],
#             y1 = line3[3],
#             line = dict(
#                 color = 'rgb(200, 200, 200)',
#                 width = 2,
#                 dash = "dash",
#             )
# )
# fig.add_shape(
#         # Case doubles every 6 days
#             type="line",
#             x0 = line6[0],
#             y0 = line6[1],
#             x1 = line6[2],
#             y1 = line6[3],
#             line = dict(
#                 color = 'rgb(200, 200, 200)',
#                 width = 2,
#                 dash = "dash",
#             )
# )
# fig.add_shape(
#         # Case doubles every 14 days
#             type="line",
#             x0 = line14[0],
#             y0 = line14[1],
#             x1 = line14[2],
#             y1 = line14[3],
#             line = dict(
#                 color = 'rgb(200, 200, 200)',
#                 width = 2,
#                 dash = "dash",
#             )
# )
# fig.add_shape(
#         # Case doubles every 1 month
#             type="line",
#             x0 = line30[0],
#             y0 = line30[1],
#             x1 = line30[2],
#             y1 = line30[3],
#             line = dict(
#                 color = 'rgb(200, 200, 200)',
#                 width = 2,
#                 dash = "dash",
#             )
# )
# ## Add annotation for reference lines 
# fig.update_layout(
#     annotations = [
#         dict(
#             x = line1[2],
#             y = line1[3],
#             # xanchor="right",
#             text = "Case doubles every day"
#         ),
#         dict(
#             x = line3[2],
#             y = line3[3],
#             text = "Case doubles every 3 days"
#         ),
#         dict(
#             x = line6[2],
#             y = line6[3],
#             text = "Case doubles every 6 days"
#         ),
#         dict(
#             x = line14[2],
#             y = line14[3],
#             text = "Case doubles every 2 weeks"
#         ),
#         dict(
#             x = line30[2],
#             y = line30[3],
#             text = "Case doubles every 1 month"
#         )
#     ]
# )

# fig.add_trace(go.Scatter(
#     x=x_list,
#     y=y_list,
#     mode="text",
#     name="Country name",
#     text=annotation_list,
#     textposition="top right",
#     textfont=dict(
#         color="crimson")
# ))
# fig.update_layout(showlegend=False)

# # Set y-axis to real number of cases (instead of log(cases)) to be more clear.
# fig.update_yaxes(
#     ticktext=["100 ", "1K ", "10K ", "100K ", "1M "],
#     tickvals=[2, 3, 4, 5, 6],
# )

# # our custom event handler
# def update_trace(trace, points, selector):
#     # this list stores the points which were clicked on
#     # in all but one event they it be empty
#     if len(points.point_inds) > 0:
#         for i in range( len(fig.data) ):
#             #fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)
#             if i == points.trace_index:
#                 fig.data[i]['line']['color'] = 'red'
#                 fig.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta
#             else:
#                 fig.data[i]['line']['color'] = 'rgb(200, 200, 200)'
#                 fig.data[i]['line']['width'] = default_linewidth
                
# # we need to add the on_click event to each trace separately       
# for i in range( len(fig.data) ):
#     fig.data[i].on_click(update_trace)

# # let's show the figure 
# fig

**--- NOTE ABOUT HOW TO CALCULATE THE REFERENCE LINE FOR GRAPH ---**

Set threshold = 50 (for 50 cases at day 0)

**1. Number of cases doubles every day:**

x = 0, 1, 2, ...

y = 2^(x + log2(threshold))
=> The result of y would be: 50, 100, 200, 400, 800, ...

**2. Number of cases doubles every 2 days:**

x = 0, 1, 2...

y = 2^(x/2 + log2(threshold))
=> The result of y would be: 50, 71, 100, 141, 200, 282, 400....

**3. Number of cases doubles every month (30 days):**

x = 0, 1, 2...

y = 2^(x/30 + log2(threshold))
=> The result of y would be: 50,51,17, 52.36, 53.59, ...

**G. FURTHER READING**

My report on Medium about the Vietnam data I worked on, combining with the world data analysis. (Coming soon)

My other kernels about the same topic: https://www.kaggle.com/nhntran/covid-19-vietnam-data-eda-and-visualization?scriptVersionId=32963257


**If this kernel helps you, please upvote. Thank you! **